# Análise Exploratória de Dados de Vendas Online

## Por: Kenzo, Maurício Terreo, e Vinícius Leôncio

### Módulo: Técnicas de Programação I

---

O objetivo deste projeto é proporcionar a oportunidade de aplicar os conhecimentos adquiridos ao longo da disciplina de Técnicas de Programação 1 em um contexto prático, relevante e data-driven. Com uma análise exploratória de dados de vendas online, utilizando um conjunto de dados real, a fim de extrair insights e entender melhor a base de dados.

Base de dados usada: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?select=olist_order_items_dataset.csv

Foi explorado os seguinte dataset nesse arquivo:

- olist_order_items_dataset.csv
- olist_products_dataset.csv

---


## Bibliotecas usadas


In [1]:
import numpy as np
import pandas as pd

## Importação dos dados


In [13]:
orders = pd.read_csv('olist_order_items_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')

## Order items


#### Informações


In [3]:
print(orders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
None


São 112650 dados, onde nenhuma coluna possui valores nulos. A única coluna que precisa ter o tipo corrigido é 'shipping_limit_date', pois está sendo interpretada como string apesar de representar uma data, isso pode ser feito com a função to_datetime() do próprio pandas. As outras colunas ou estão corretas ou já serão excluídas pois não serão utilizadas.


In [14]:
print(type(orders['shipping_limit_date'][0]))
orders['shipping_limit_date'] = pd.to_datetime(orders['shipping_limit_date'])
print(type(orders['shipping_limit_date'][0]))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


#### Descrição


In [6]:
print(orders[['shipping_limit_date','price','freight_value']].describe())

                 shipping_limit_date          price  freight_value
count                         112650  112650.000000  112650.000000
mean   2018-01-07 15:36:52.192685312     120.653739      19.990320
min              2016-09-19 00:15:34       0.850000       0.000000
25%       2017-09-20 20:57:27.500000      39.900000      13.080000
50%              2018-01-26 13:59:35      74.990000      16.260000
75%    2018-05-10 14:34:00.750000128     134.900000      21.150000
max              2020-04-09 22:35:08    6735.000000     409.680000
std                              NaN     183.633928      15.806405


#### Dados


In [8]:
orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [7]:
orders.tail()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72
112649,fffe41c64501cc87c801fd61db3f6244,1,350688d9dc1e75ff97be326363655e01,f7ccf836d21b2fb1de37564105216cc1,2018-06-12 17:10:13,43.00,12.79


Nessa tabela possuimos o preço do pedido e o valor de frete, podemos somar esses valores e criar uma coluna com o preço total.


In [16]:
final_price = np.array(0 for i in range(orders.shape[0]))

final_price = orders['price'] + orders['freight_value']

orders['final_price'] = final_price
orders.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,final_price
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,218.04


As colunas 'order_id','order_item_id' e 'seller_id' representam, respectivamente: o identificador do pedido, o identificador do produto do pedido e o identificador do vendedor. Esses valores não serão utilizados, então excluímos ela com a função drop().


In [17]:
orders = orders.drop(['order_id','order_item_id','seller_id'], axis=1)
orders.head()

,product_id,shipping_limit_date,price,freight_value,final_price
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,58.90,13.29,72.19
1,e5f2d52b802189ee658865ca93d83a8f,2017-05-03 11:05:13,239.90,19.93,259.83
2,c777355d18b72b67abbeef9df44fd0fd,2018-01-18 14:48:30,199.00,17.87,216.87
3,7634da152a4610f1595efa32f14722fc,2018-08-15 10:10:18,12.99,12.79,25.78
4,ac6c3623068f30de03045865e4e10089,2017-02-13 13:57:51,199.90,18.14,218.04


## Products


### Informações


In [12]:
print(products.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
None


São 32951 dados no total, sendo que todas as colunas, fora product_id, possuem algum valor nulo.


Os nulos para o caso as colunas de peso, altura, largura e profundidade do produto são as seguintes linhas:


In [23]:
nulos = products.loc[(products['product_weight_g'].isnull() )& (products['product_length_cm'].isnull()) & (products['product_height_cm'].isnull()) & (products['product_width_cm'].isnull())]
nulos


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.0,865.0,3.0,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Para o caso da linha 8578, nós temos o nome da categoria daquele produto, então podemos completar esses valores com a mediana dos produtos da mesma categoria:


In [30]:
median_values = products[products['product_category_name'] == 'bebes'][['product_length_cm','product_height_cm','product_width_cm']].median()

products.loc[products['product_weight_g'].isnull() & ~products['product_category_name'].isnull()] = median_values['product_length_cm']
products.loc[products['product_weight_g'].isnull() & ~products['product_category_name'].isnull()] = median_values['product_height_cm']
products.loc[products['product_weight_g'].isnull() & ~products['product_category_name'].isnull()] = median_values['product_width_cm']

Para o caso da coluna 'product_category_name' poderíamos induzir seus valores a partir de uma relação entre suas medidas e sua categoria. Primeiro deveríamos encontrar se existe essa relação e , por exemplo, poderíamos treinar um algoritmo de KNN para classificar esses dados, por isso fugir do escopo desse trabalho isso não foi feito.


In [16]:
products.describe()

,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
count,32341.000000,32341.000000,32341.000000,32949.000000,32949.000000,32949.000000,32949.000000
mean,48.476949,771.495285,2.188986,2276.472488,30.815078,16.937661,23.196728
std,10.245741,635.115225,1.736766,4282.038731,16.914458,13.637554,12.079047
min,5.000000,4.000000,1.000000,0.000000,7.000000,2.000000,6.000000
25%,42.000000,339.000000,1.000000,300.000000,18.000000,8.000000,15.000000
50%,51.000000,595.000000,1.000000,700.000000,25.000000,13.000000,20.000000
75%,57.000000,972.000000,3.000000,1900.000000,38.000000,21.000000,30.000000
max,76.000000,3992.000000,20.000000,40425.000000,105.000000,105.000000,118.000000


#### Dados


In [17]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [32]:
products.tail()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0
32950,106392145fca363410d287a815be6de4,cama_mesa_banho,58.0,309.0,1.0,2083.0,12.0,2.0,7.0


A tabela products possui o atributo 'product_category_name', que pode assumir 73 valores distintos(ignorando os valores nulos):


In [33]:
print(f"Qnt: {products['product_category_name'].nunique()}")
print(f"Categorias: {products['product_category_name'].unique()}")

Qnt: 74
Categorias: ['perfumaria' 'artes' 'esporte_lazer' 'bebes' 'utilidades_domesticas'
 'instrumentos_musicais' 'cool_stuff' 'moveis_decoracao'
 'eletrodomesticos' 'brinquedos' 'cama_mesa_banho'
 'construcao_ferramentas_seguranca' 'informatica_acessorios'
 'beleza_saude' 'malas_acessorios' 'ferramentas_jardim'
 'moveis_escritorio' 'automotivo' 'eletronicos' 'fashion_calcados'
 'telefonia' 'papelaria' 'fashion_bolsas_e_acessorios' 'pcs'
 'casa_construcao' 'relogios_presentes'
 'construcao_ferramentas_construcao' 'pet_shop' 'eletroportateis'
 'agro_industria_e_comercio' nan 'moveis_sala' 'sinalizacao_e_seguranca'
 'climatizacao' 'consoles_games' 'livros_interesse_geral'
 'construcao_ferramentas_ferramentas' 'fashion_underwear_e_moda_praia'
 'fashion_roupa_masculina'
 'moveis_cozinha_area_de_servico_jantar_e_jardim'
 'industria_comercio_e_negocios' 'telefonia_fixa'
 'construcao_ferramentas_iluminacao' 'livros_tecnicos'
 'eletrodomesticos_2' 'artigos_de_festas' 'bebidas' 'market_place'


Contudo, verificando de perto, podemos ver que existe uma categoria marcada como 33.0:


In [37]:
exemplo = products[(products['product_category_name'] == 33.0)]
exemplo

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0,33.0


Claramente essa linha não representa um dado real e deve ser excluída.


In [38]:
products = products.drop(products[(products['product_category_name'] == 33.0)].index)
exemplo = products[(products['product_category_name'] == 33.0)]
exemplo

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


Frequência absoluta das categorias:


In [40]:
products['product_category_name'].value_counts()

product_category_name
cama_mesa_banho                  3029
esporte_lazer                    2867
moveis_decoracao                 2657
beleza_saude                     2444
utilidades_domesticas            2335
                                 ... 
fashion_roupa_infanto_juvenil       5
casa_conforto_2                     5
pc_gamer                            3
seguros_e_servicos                  2
cds_dvds_musicais                   1
Name: count, Length: 73, dtype: int64

Frequência relativa:


In [41]:
products['product_category_name'].value_counts(normalize=True)

product_category_name
cama_mesa_banho                  0.093661
esporte_lazer                    0.088652
moveis_decoracao                 0.082158
beleza_saude                     0.075572
utilidades_domesticas            0.072202
                                   ...   
fashion_roupa_infanto_juvenil    0.000155
casa_conforto_2                  0.000155
pc_gamer                         0.000093
seguros_e_servicos               0.000062
cds_dvds_musicais                0.000031
Name: proportion, Length: 73, dtype: float64

A partir da largura, altura e profundidade do produto podemos criar uma coluna contendo o volume daquele produto:


In [42]:
products['product_volume_cm3'] = products['product_length_cm'] * products['product_height_cm'] * products['product_width_cm']
products['product_volume_cm3'].describe()

count     32949.000000
mean      16564.096695
std       27057.041650
min         168.000000
25%        2880.000000
50%        6840.000000
75%       18480.000000
max      296208.000000
Name: product_volume_cm3, dtype: float64

Usando os valores existentes como referêncial para classificar o tamanho dos produtos, podemos usar os quartis para classificar como 'Grande', 'Médio' ou 'Pequeno'. Para no caso de ser médio, precisamos colocar a condição ('Médio' if v<18480 and v> 2880 else np.NaN), pois caso usado apenas o else, os produtos com valores nulos de medias seriam classificados como Médios.


In [43]:
cond = lambda v: 'Grande' if v>=18480 else ('Pequeno' if v<=2880 else ('Médio' if v<18480 and v> 2880 else np.NaN))
products['product_volume_category'] = products['product_volume_cm3'].apply(cond)

In [45]:
products[['product_id','product_category_name','product_volume_category']].head()

,product_id,product_category_name,product_volume_category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,Pequeno
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,Médio
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,Pequeno
3,cef67bcfe19066a932b7673e239eb23d,bebes,Pequeno
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,Médio


Por fim, podemos excluir as colunas 'product_name_lenght','product_description_lenght','product_photos_qty' e 'product_weight_g' pois não serão usadas nesse caso.


In [47]:
products= products.drop(['product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g'],axis=1)
products.head()

,product_id,product_category_name,product_length_cm,product_height_cm,product_width_cm,product_volume_cm3,product_volume_category
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,16.0,10.0,14.0,2240.0,Pequeno
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,30.0,18.0,20.0,10800.0,Médio
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,18.0,9.0,15.0,2430.0,Pequeno
3,cef67bcfe19066a932b7673e239eb23d,bebes,26.0,4.0,26.0,2704.0,Pequeno
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,20.0,17.0,13.0,4420.0,Médio


## Relacionando as duas tabelas


Usando .merge() podemos juntar as duas tabelas a partir da coluna product_id que as duas possuem em comum, caso não especificado, ela funciona como um LEFT JOIN do SQL.


In [48]:
orders_products = pd.merge(orders,products, on='product_id')
orders_products.head()

,product_id,shipping_limit_date,price,freight_value,final_price,product_category_name,product_length_cm,product_height_cm,product_width_cm,product_volume_cm3,product_volume_category
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,58.9,13.29,72.19,cool_stuff,28.0,9.0,14.0,3528.0,Médio
1,4244733e06e7ecb4970a6e2683c13e61,2017-07-05 02:44:11,55.9,17.96,73.86,cool_stuff,28.0,9.0,14.0,3528.0,Médio
2,4244733e06e7ecb4970a6e2683c13e61,2018-05-23 10:56:25,64.9,18.33,83.23,cool_stuff,28.0,9.0,14.0,3528.0,Médio
3,4244733e06e7ecb4970a6e2683c13e61,2017-08-07 18:55:08,58.9,16.17,75.07,cool_stuff,28.0,9.0,14.0,3528.0,Médio
4,4244733e06e7ecb4970a6e2683c13e61,2017-08-16 22:05:11,58.9,13.29,72.19,cool_stuff,28.0,9.0,14.0,3528.0,Médio


In [49]:
orders_products.tail()

,product_id,shipping_limit_date,price,freight_value,final_price,product_category_name,product_length_cm,product_height_cm,product_width_cm,product_volume_cm3,product_volume_category
112644,4cc4d02efc8f249c13355147fb44e34d,2018-07-18 20:23:55,129.9,51.20,181.10,ferramentas_jardim,35.0,12.0,22.0,9240.0,Médio
112645,b10ecf8e33aaaea419a9fa860ea80fb5,2018-08-21 10:10:11,99.0,13.52,112.52,moveis_decoracao,37.0,30.0,20.0,22200.0,Grande
112646,dd469c03ad67e201bc2179ef077dcd48,2017-06-07 17:05:23,736.0,20.91,756.91,relogios_presentes,19.0,9.0,15.0,2565.0,Pequeno
112647,bbe7651fef80287a816ead73f065fc4b,2017-12-22 17:31:42,229.9,44.02,273.92,esporte_lazer,60.0,15.0,15.0,13500.0,Médio
112648,350688d9dc1e75ff97be326363655e01,2018-06-12 17:10:13,43.0,12.79,55.79,cama_mesa_banho,30.0,3.0,19.0,1710.0,Pequeno


## Perguntas


### Quais são os valores ganhos das vendas de produtos que possuem uma das 3 categorias mais frequentes?


In [26]:
orders_products['product_category_name'].value_counts()

product_category_name
cama_mesa_banho                  11115
beleza_saude                      9670
esporte_lazer                     8641
moveis_decoracao                  8334
informatica_acessorios            7827
                                 ...  
cds_dvds_musicais                   14
la_cuisine                          14
pc_gamer                             9
fashion_roupa_infanto_juvenil        8
seguros_e_servicos                   2
Name: count, Length: 74, dtype: int64

In [27]:
mask = (orders_products['product_category_name'] == 'cama_mesa_banho') | (orders_products['product_category_name'] == 'beleza_saude') | (orders_products['product_category_name'] == 'esporte_lazer')
orders_products[mask][['product_category_name','price']].groupby('product_category_name').sum()

,price
product_category_name,
beleza_saude,1258681.34
cama_mesa_banho,1036988.68
esporte_lazer,988048.97


### Quais são os produtos com o preço acima da média **por categoria**?


In [28]:
mask_geral = np.array([False for i in range(112650)])
# Fazemos uma mask para cada categoria e juntamos a mask_geral
# dessa forma não precisamos escrever uma condição para cada uma das 73 categorias na mão
for categoria in orders_products['product_category_name'].value_counts().keys():
        mask = orders_products[orders_products['product_category_name'] == categoria]['price'] > orders_products[orders_products['product_category_name'] == categoria]['price'].mean()
        for i in np.where(mask):
                mask_geral[i] = True

In [29]:
orders_products[mask_geral]

,product_id,shipping_limit_date,price,freight_value,final_price,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_volume_cm3,product_volume_category
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,58.9,13.29,72.19,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,3528.0,Médio
1,4244733e06e7ecb4970a6e2683c13e61,2017-07-05 02:44:11,55.9,17.96,73.86,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,3528.0,Médio
2,4244733e06e7ecb4970a6e2683c13e61,2018-05-23 10:56:25,64.9,18.33,83.23,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,3528.0,Médio
3,4244733e06e7ecb4970a6e2683c13e61,2017-08-07 18:55:08,58.9,16.17,75.07,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,3528.0,Médio
4,4244733e06e7ecb4970a6e2683c13e61,2017-08-16 22:05:11,58.9,13.29,72.19,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,3528.0,Médio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11103,b8a0d73b2a06e7910d9864dccdb0cda2,2018-03-22 23:07:44,59.9,17.02,76.92,beleza_saude,43.0,295.0,1.0,371.0,20.0,16.0,16.0,5120.0,Médio
11104,b8a0d73b2a06e7910d9864dccdb0cda2,2018-03-08 23:15:32,59.9,16.86,76.76,beleza_saude,43.0,295.0,1.0,371.0,20.0,16.0,16.0,5120.0,Médio
11110,b8a0d73b2a06e7910d9864dccdb0cda2,2018-04-19 12:50:13,69.9,17.08,86.98,beleza_saude,43.0,295.0,1.0,371.0,20.0,16.0,16.0,5120.0,Médio
11112,b8a0d73b2a06e7910d9864dccdb0cda2,2018-03-15 04:08:38,59.9,22.13,82.03,beleza_saude,43.0,295.0,1.0,371.0,20.0,16.0,16.0,5120.0,Médio


### Qual a categoria de produtos que mais foi entregue no ano de 2017? E qual o produto?

Adendo: nos datasets desse arquivo, nós temos acesso a informação 'shipping_limit_date' que é a expectiva limite de entrega do produto. Poderíamos ter pego o dataset orders, onde possui a data específica de entrega e feito um merge com nossa tabela.


In [50]:
mask = (orders_products['shipping_limit_date'] > '2017-01-01 00:00:01') & (orders_products['shipping_limit_date'] < '2018-01-01 00:00:01')
orders_products[mask]['product_category_name'].value_counts()

product_category_name
cama_mesa_banho                5118
moveis_decoracao               4058
esporte_lazer                  4038
beleza_saude                   3590
informatica_acessorios         3042
                               ... 
pc_gamer                          4
portateis_casa_forno_e_cafe       4
artes_e_artesanato                2
seguros_e_servicos                2
fraldas_higiene                   2
Name: count, Length: 72, dtype: int64

Obtendo o id do produto mais vendido:


In [51]:
orders_products[mask]['product_id'].value_counts()

product_id
99a4788cb24856965c36a24e339b6058    355
422879e10f46682990de24d770e7f83d    264
154e7e31ebfa092203795c972e5804a6    220
389d119b48cf3043d311335e499d9c6b    208
53759a2ecddad2bb87a079a1f1519f73    204
                                   ... 
fd0f7402731790468090d7895cf61c0f      1
e84bf31698e5842d605c5aa9cfc8b02a      1
1002287b975c051e4f283579598cf6d2      1
c17abb4b4009c10138dec784ec61ba63      1
bbe7651fef80287a816ead73f065fc4b      1
Name: count, Length: 17002, dtype: int64

### Qual a categoria majoritária dos produtos tidos como 'Grandes'?


In [52]:
mask = orders_products['product_volume_category'] == 'Grande'
orders_products[mask]['product_category_name'].value_counts()

product_category_name
cama_mesa_banho           3701
utilidades_domesticas     2917
ferramentas_jardim        2474
moveis_decoracao          2419
esporte_lazer             1633
                          ... 
artes_e_artesanato           2
musica                       2
flores                       2
livros_interesse_geral       2
fraldas_higiene              1
Name: count, Length: 67, dtype: int64